<a href="https://colab.research.google.com/github/MarinaWyss/search-engine-thesis/blob/master/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers

     |████████████████████████████████| 501kB 2.8MB/s 
     |████████████████████████████████| 1.0MB 45.9MB/s 
     |████████████████████████████████| 3.7MB 35.9MB/s 
     |████████████████████████████████| 870kB 39.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=7be4a6aa387b6a6443dd96841f6091ea7b1bef1c99b10d2cb3bd5f191157c298
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [4]:
import pandas as pd
import numpy as np
import random
import torch
import transformers as ppb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression

In [5]:
# load data
from google.colab import files
upload = files.upload()

Saving textBlockBefore.csv to textBlockBefore.csv


In [0]:
import io
full_data = pd.read_csv(io.BytesIO(upload["textBlockBefore.csv"]))

In [0]:
# clean data
full_data["text"] = full_data["text"].apply(lambda x: x.replace("?.", "?"))

In [0]:
full_data_vote = full_data.dropna()

In [0]:
# importing model and tokenizer
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

In [40]:
# load model and tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [0]:
# tokenization
tokenized = full_data_vote["text"].apply((lambda x: tokenizer.encode(x, add_special_tokens = True, max_length = 511)))

In [0]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len- len(i)) for i in tokenized.values])

In [0]:
# BERT model
attention_mask = np.where(padded != 0, 1, 0)

In [0]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

In [0]:
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask = attention_mask)

In [0]:
# extract features and labels
features = last_hidden_states[0][:,0,:].numpy()
labels = full_data_vote["label"]

In [0]:
# train test split
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [64]:
# basic logistic regression model
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [65]:
# accuracy
lr_clf.score(test_features, test_labels)

0.5202702702702703